In [ ]:
import qiskit
import numpy as np
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit, assemble, transpile
from qiskit.visualization import plot_histogram
from qiskit import IBMQ, Aer
import matplotlib.pyplot as plt
from qiskit.providers.aer import AerSimulator
from qiskit.circuit.library import MCMT

In [ ]:
#input length of circuit
w = '10'
n = len(w)

In [ ]:
#define grover algorithm
def grover_oracle_2(n,b):
    grover_circ = QuantumCircuit(n)
    if(b == '00'):
        grover_circ.cz(0,1)
        grover_circ.x(0)
        grover_circ.x(1)
    if(b == '01'):
        grover_circ.cz(0,1)
        grover_circ.x(1)
    if(b == '10'):
        grover_circ.cz(0,1)
        grover_circ.x(0)
    if(b == '11'):
        grover_circ.cz(0,1)
        
    circ = grover_circ.to_gate()
    circ.name = "Grovers Oracle"
    
    return circ 
        

In [ ]:
#define full circuit 
def grovers_algorithm_2(oracle,n):
    circuit = QuantumCircuit(n,n)

    #first apply Hadamard on both gates
    for i in range(n):
        circuit.h(i)


    #append oracle to circuit
    circuit.append(oracle,range(n))

    #apply diffuser
    circuit.h([0,1])
    circuit.z([0,1])
    circuit.cz(0,1)
    circuit.h([0,1])

    #measure
    for i in range(n):
        circuit.measure(i,i)

    return circuit

In [ ]:
def grover_oracle(n,w):
    #define grover algorithm oracles
    w = w[::-1]
    circ = QuantumCircuit(n)
    #define for each case of  qubits the different oracles
    #apply x-gate for w[i]=='0'
    for i in range(n):
        if(w[i]=='0'):
            circ.x(i)
            
    #multi-controlled multi-target gate, for an arbitrary singly controlled target gate.
    #using z-gate
    circ += MCMT('z', n-1, 1) 
    
    #re-apply x-gate for w[i]=='0'
    for i in range(n):
        if(w[i]=='0'):
            circ.x(i)
    return circ
        

In [ ]:
def diffuser(n):
    c = QuantumCircuit(n)
    for i in range(n):
        c.x(i)
        
    c += MCMT('z', n-1, 1)
    
    for i in range(n):
        c.x(i)
    return c

In [ ]:
def grovers_algorithm(oracle,n):
    #define full circuit 
    grover_circ = QuantumCircuit(n)

    #first apply Hadamard on both gates
    for i in range(n):
        grover_circ.h(i)
    grover_circ.barrier()
    #apply oracle
    grover_circ.append(oracle, range(n))
    grover_circ.barrier()
    #apply hadamard
    for i in range(n):
        grover_circ.h(i)
    grover_circ.barrier()
    #apply diffuser
    grover_circ += diffuser(n)
    grover_circ.barrier()
    
    for i in range(n):
        grover_circ.h(i)
    #measure
    grover_circ.measure_all()
    
    return grover_circ

In [ ]:
oracle_gate = grover_oracle(7,'0000100')
grovers_circuit = grovers_algorithm(oracle_gate,7)
grovers_circuit.draw()

In [ ]:
#noisless simulations

simulator = Aer.get_backend('aer_simulator')
circ = transpile(grovers_circuit, simulator)

# Run and get counts
result = simulator.run(circ).result()
counts = result.get_counts(circ)

plot_histogram(counts, title="aer_simulator for Grovers")

In [ ]:
#use of noisy simulator
from qiskit.test.mock import FakeVigo
device_backend = FakeVigo()

# Transpile the circuit for the noisy basis gates
sim_vigo = AerSimulator.from_backend(device_backend)
tcirc = transpile(grovers_circuit, sim_vigo)

# Execute noisy simulation and get counts
result_noise = sim_vigo.run(tcirc).result()
counts_noise = result_noise.get_counts(0)
plot_histogram(counts_noise,title="Virgo noisy simulator for Grovers")


In [ ]:
from qiskit.test.mock import FakeVigo,FakeCasablanca
device_backend = FakeCasablanca()

# Transpile the circuit for the noisy basis gates
sim_casa = AerSimulator.from_backend(device_backend)
tcirc = transpile(grovers_circuit, sim_vigo)

# Execute noisy simulation and get counts
result_noise = sim_casa.run(tcirc).result()
counts_noise = result_noise.get_counts(0)
plot_histogram(counts_noise,
               title="Casablanca noisy simulator for Grovers")


In [ ]:
import time
import matplotlib.pyplot as plt

#calculate execution time for different oracles 

time_for_diff_oracles=[]
for i in range(20):
    start_time = time.time()
    n=2
    b = np.random.randint(1,2**n)
    b_str = format(b, '0'+str(n)+'b')
    oracle_gate = grover_oracle(n,b_str)
    bv_circuit = grovers_algorithm(oracle_gate, n)
    time_for_diff_oracles.append(time.time()-start_time)
    
plt.plot(range(20),time_for_diff_oracles,"o")
plt.xlabel("Oracle number (# qubits passed in oracle=2)")
plt.ylabel("Execusion time (s)")
plt.savefig("oracle_execusion_grover.pdf")
plt.show()

In [ ]:
#calculate execution time for different number of input qubits

time_for_diff_n=[]
numbers=[3,4,5,6,7,8,9,10,11,12,13]
for i in numbers:
    start_time = time.time()
    n=i
    b = np.random.randint(1,2**n)
    b_str = format(b, '0'+str(n)+'b')
    oracle_gate = grover_oracle(n,b_str)
    grovers_circuit = grovers_algorithm(oracle_gate,n)
    time_for_diff_n.append(time.time()-start_time)
    
plt.plot(numbers,time_for_diff_n,"o")
plt.xlabel("Number of qubits passed in oracle")
plt.ylabel("Execusion time (s)") 
plt.savefig("n_qubit_execusion_grover.pdf")
plt.show()